In [2]:
!pip install git+https://github.com/akamhy/waybackpy.git

  Cloning https://github.com/akamhy/waybackpy.git to /tmp/pip-req-build-e61sgjvv
  Created wheel for waybackpy: filename=waybackpy-2.4.2-py3-none-any.whl size=21204 sha256=4b3aa9dbc2ff63c88d8868d3d07ecd526cf5e59f5aac1919293bb5319de1b3e4
  Stored in directory: /tmp/pip-ephem-wheel-cache-am6lhmjm/wheels/82/0d/a7/6c38a890cf03c2cae59ff65b5b128f82054db3c373179e6d95
Successfully built waybackpy


In [3]:
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import requests
import waybackpy
import pandas as pd

user_agent = "Mozilla/5.0 (Windows NT 5.1; rv:40.0) Gecko/20100101 Firefox/40.0"

In [4]:
#2018 scraper

def get_score(soup):
    critic = soup.find('span', {'class': "meter-value superPageFontColor"})
    audience = soup.find('div', {'class': "audience-score meter"}).find('span', {'class': "superPageFontColor"})
    return [critic.text, audience.text]

In [5]:
#2019-21 scraper
def getScore(soup):
    temp = soup.find_all('div', class_='mop-ratings-wrap__half')
    try:
        critic = temp[0].text.strip().replace('\n', '').split(' ')[0] #2020 scraper        
        if len(temp) > 1:
            audience = temp[1].text.strip().replace('\n', '').split(' ')[0]
        else:
            audience = "Coming"
    except:
        scores = soup.find("score-board") #2021 scraper
        return [scores["tomatometerscore"], scores["audiencescore"]]
    return [critic, audience]

In [12]:
#get number of critic and audience reviews
def getNumReviews(soup):        
    critic = soup.find_all("small", class_="mop-ratings-wrap__text--small")
    audience = soup.find_all("strong", class_="mop-ratings-wrap__text--small")
    if critic:
        critic = critic[0].text.replace("\n", '').strip()
    else:
        critic = soup.find_all("a", class_='scoreboard__link scoreboard__link--tomatometer')
        critic = critic[0].text
    if len(audience) > 1:
        audience = audience[1].text.replace("Verified Ratings: ", '')
    else:
        audience = soup.find_all("a", class_='scoreboard__link scoreboard__link--audience')
        if audience:
            audience = audience[0].text
        else:    
            audience = "Coming"
    return [critic, audience]

In [7]:
movie_urls = {'Wonder Woman 1984': 'https://www.rottentomatoes.com/m/wonder_woman_1984/', 
              'Soul':'https://www.rottentomatoes.com/m/soul_2020/', 
              'Mulan': 'https://www.rottentomatoes.com/m/mulan_2020/',
              'Birds of Prey': 'https://www.rottentomatoes.com/m/birds_of_prey_2020/',
              'Sonic': 'https://www.rottentomatoes.com/m/sonic_the_hedgehog_2020/'
             }

movie_urls["Captain Marvel"] = "https://www.rottentomatoes.com/m/captain_marvel/"
movie_urls["Lion King"] = "https://www.rottentomatoes.com/m/the_lion_king_2019/"
movie_urls["Aladdin"] = "https://www.rottentomatoes.com/m/aladdin/"
movie_urls["Joker"] = "https://www.rottentomatoes.com/m/joker_2019/"
movie_urls["Shazam!"] = "https://www.rottentomatoes.com/m/shazam/"
movie_urls["Godzilla: King of the Monsters"] = "https://www.rottentomatoes.com/m/godzilla_king_of_the_monsters_2019/"

movie_urls['Tenet'] = 'https://www.rottentomatoes.com/m/tenet/'
movie_urls['Scoob'] = 'https://www.rottentomatoes.com/m/scoob/'
movie_urls['The Marksman'] = 'https://www.rottentomatoes.com/m/the_marksman_2021/'
movie_urls['Artemis Fowl'] = 'https://www.rottentomatoes.com/m/artemis_fowl/'
movie_urls['Lovebirds'] = 'https://www.rottentomatoes.com/m/the_lovebirds_2020/'

In [8]:
#for 2019-21
def build(key, date1, date2):
    dates = pd.date_range(date1, date2).tolist()
    wayback = waybackpy.Url(movie_urls[key], user_agent)
    scores = []
    for date in dates:
        try:
            archived = wayback.near(year=date.year, month=date.month, day=date.day).archive_url
        except:
            print(date)
            continue
        page = requests.get(archived).text
        soup = BeautifulSoup(page, 'lxml')
        scores.append(getScore(soup) + getNumReviews(soup) + [date, key])
    return scores

In [9]:
#for 2017-18
def build2(key, date1, date2):
    dates = pd.date_range(date1, date2).tolist()
    wayback = waybackpy.Url(movie_urls[key], user_agent)
    scores = []
    for date in dates:
        try:
            archived = wayback.near(year=date.year, month=date.month, day=date.day).archive_url
        except:
            print(date)
            continue
        page = requests.get(archived).text
        soup = BeautifulSoup(page, 'lxml')
        scores.append(get_score(soup) + getNumReviews(soup) + [date, key])
    return scores

In [10]:
joker = build("Joker", '2019-09-03', '2020-09-03')
joker

2019-11-28 00:00:00
2019-11-29 00:00:00
2019-12-05 00:00:00
2019-12-15 00:00:00
2019-12-21 00:00:00
2020-01-11 00:00:00
2020-02-04 00:00:00
2020-02-12 00:00:00
2020-02-13 00:00:00
2020-02-19 00:00:00
2020-02-20 00:00:00
2020-02-21 00:00:00
2020-03-04 00:00:00
2020-03-11 00:00:00
2020-03-12 00:00:00
2020-03-19 00:00:00
2020-03-22 00:00:00
2020-03-23 00:00:00
2020-03-27 00:00:00
2020-03-29 00:00:00
2020-04-14 00:00:00
2020-07-12 00:00:00


[['85%',
  'Coming',
  '40',
  'Ratings: Not yet available',
  Timestamp('2019-09-03 00:00:00', freq='D'),
  'Joker'],
 ['83%',
  'Coming',
  '42',
  'Ratings: Not yet available',
  Timestamp('2019-09-04 00:00:00', freq='D'),
  'Joker'],
 ['85%',
  'Coming',
  '46',
  'Ratings: Not yet available',
  Timestamp('2019-09-05 00:00:00', freq='D'),
  'Joker'],
 ['85%',
  'Coming',
  '46',
  'Ratings: Not yet available',
  Timestamp('2019-09-06 00:00:00', freq='D'),
  'Joker'],
 ['85%',
  'Coming',
  '46',
  'Ratings: Not yet available',
  Timestamp('2019-09-07 00:00:00', freq='D'),
  'Joker'],
 ['86%',
  'Coming',
  '49',
  'Ratings: Not yet available',
  Timestamp('2019-09-08 00:00:00', freq='D'),
  'Joker'],
 ['86%',
  'Coming',
  '49',
  'Ratings: Not yet available',
  Timestamp('2019-09-09 00:00:00', freq='D'),
  'Joker'],
 ['86%',
  'Coming',
  '50',
  'Ratings: Not yet available',
  Timestamp('2019-09-10 00:00:00', freq='D'),
  'Joker'],
 ['77%',
  'Coming',
  '75',
  'Ratings: Not yet

In [14]:
captain = build("Captain Marvel", '2019-03-06', '2020-03-06')
captain

2019-07-05 00:00:00
2019-07-24 00:00:00
2019-09-05 00:00:00
2019-10-17 00:00:00
2019-11-17 00:00:00
2019-12-06 00:00:00
2019-12-14 00:00:00


[['84%',
  'Audience',
  '170',
  'User Ratings: ',
  Timestamp('2019-03-06 00:00:00', freq='D'),
  'Captain Marvel'],
 ['82%',
  'Audience',
  '229',
  'User Ratings: ',
  Timestamp('2019-03-07 00:00:00', freq='D'),
  'Captain Marvel'],
 ['81%',
  '35%',
  '289',
  'liked it',
  Timestamp('2019-03-08 00:00:00', freq='D'),
  'Captain Marvel'],
 ['79%',
  '53%',
  '333',
  'liked it',
  Timestamp('2019-03-09 00:00:00', freq='D'),
  'Captain Marvel'],
 ['80%',
  '57%',
  '350',
  'liked it',
  Timestamp('2019-03-10 00:00:00', freq='D'),
  'Captain Marvel'],
 ['80%',
  '59%',
  '378',
  'liked it',
  Timestamp('2019-03-11 00:00:00', freq='D'),
  'Captain Marvel'],
 ['80%',
  '61%',
  '395',
  'liked it',
  Timestamp('2019-03-12 00:00:00', freq='D'),
  'Captain Marvel'],
 ['79%',
  '63%',
  '406',
  'liked it',
  Timestamp('2019-03-13 00:00:00', freq='D'),
  'Captain Marvel'],
 ['79%',
  '62%',
  '406',
  'liked it',
  Timestamp('2019-03-14 00:00:00', freq='D'),
  'Captain Marvel'],
 ['79%

In [15]:
lion = build("Lion King", '2019-07-11', '2020-07-11')
lion

[['57%',
  'Coming',
  '82',
  'Ratings: Not yet available',
  Timestamp('2019-07-11 00:00:00', freq='D'),
  'Lion King'],
 ['60%',
  'Coming',
  '120',
  'Ratings: Not yet available',
  Timestamp('2019-07-12 00:00:00', freq='D'),
  'Lion King'],
 ['60%',
  'Coming',
  '122',
  'Ratings: Not yet available',
  Timestamp('2019-07-13 00:00:00', freq='D'),
  'Lion King'],
 ['59%',
  'Coming',
  '123',
  'Ratings: Not yet available',
  Timestamp('2019-07-14 00:00:00', freq='D'),
  'Lion King'],
 ['61%',
  'Coming',
  '142',
  'Ratings: Not yet available',
  Timestamp('2019-07-15 00:00:00', freq='D'),
  'Lion King'],
 ['61%',
  'Coming',
  '142',
  'Ratings: Not yet available',
  Timestamp('2019-07-16 00:00:00', freq='D'),
  'Lion King'],
 ['58%',
  'Coming',
  '172',
  'Ratings: Not yet available',
  Timestamp('2019-07-17 00:00:00', freq='D'),
  'Lion King'],
 ['56%',
  'Coming',
  '213',
  'Ratings: Not yet available',
  Timestamp('2019-07-18 00:00:00', freq='D'),
  'Lion King'],
 ['55%',


In [16]:
aladdin = build("Aladdin", '2019-05-23', '2020-05-23')
aladdin

2019-06-30 00:00:00
2019-08-03 00:00:00
2019-09-08 00:00:00
2020-03-28 00:00:00


[['61%',
  'Coming',
  '138',
  'Coming',
  Timestamp('2019-05-23 00:00:00', freq='D'),
  'Aladdin'],
 ['59%',
  'Coming',
  '201',
  'Coming',
  Timestamp('2019-05-24 00:00:00', freq='D'),
  'Aladdin'],
 ['59%',
  'Coming',
  '201',
  'Coming',
  Timestamp('2019-05-25 00:00:00', freq='D'),
  'Aladdin'],
 ['58%',
  'Coming',
  '272',
  'Coming',
  Timestamp('2019-05-26 00:00:00', freq='D'),
  'Aladdin'],
 ['58%',
  'Coming',
  '272',
  'Coming',
  Timestamp('2019-05-27 00:00:00', freq='D'),
  'Aladdin'],
 ['58%',
  'Coming',
  '272',
  'Coming',
  Timestamp('2019-05-28 00:00:00', freq='D'),
  'Aladdin'],
 ['57%',
  'Coming',
  '288',
  'Coming',
  Timestamp('2019-05-29 00:00:00', freq='D'),
  'Aladdin'],
 ['56%',
  'Coming',
  '293',
  'Coming',
  Timestamp('2019-05-30 00:00:00', freq='D'),
  'Aladdin'],
 ['56%',
  'Coming',
  '298',
  'Coming',
  Timestamp('2019-05-31 00:00:00', freq='D'),
  'Aladdin'],
 ['56%',
  'Coming',
  '303',
  'Coming',
  Timestamp('2019-06-01 00:00:00', freq=

In [17]:
shazam = build("Shazam!", '2019-03-24', '2020-03-24')
shazam

2019-04-03 00:00:00
2019-04-09 00:00:00
2019-06-14 00:00:00
2019-09-09 00:00:00
2019-09-20 00:00:00
2019-09-21 00:00:00
2020-01-04 00:00:00
2020-01-07 00:00:00
2020-01-10 00:00:00


[['94%',
  'Audience',
  '47',
  'User Ratings: ',
  Timestamp('2019-03-24 00:00:00', freq='D'),
  'Shazam!'],
 ['92%',
  'Audience',
  '62',
  'User Ratings: ',
  Timestamp('2019-03-25 00:00:00', freq='D'),
  'Shazam!'],
 ['93%',
  'Audience',
  '69',
  'User Ratings: ',
  Timestamp('2019-03-26 00:00:00', freq='D'),
  'Shazam!'],
 ['92%',
  'Audience',
  '76',
  'User Ratings: ',
  Timestamp('2019-03-27 00:00:00', freq='D'),
  'Shazam!'],
 ['92%',
  'Audience',
  '78',
  'User Ratings: ',
  Timestamp('2019-03-28 00:00:00', freq='D'),
  'Shazam!'],
 ['93%',
  'Audience',
  '81',
  'User Ratings: ',
  Timestamp('2019-03-29 00:00:00', freq='D'),
  'Shazam!'],
 ['93%',
  'Audience',
  '91',
  'User Ratings: ',
  Timestamp('2019-03-30 00:00:00', freq='D'),
  'Shazam!'],
 ['93%',
  'Audience',
  '91',
  'User Ratings: ',
  Timestamp('2019-03-31 00:00:00', freq='D'),
  'Shazam!'],
 ['93%',
  'Audience',
  '91',
  'User Ratings: ',
  Timestamp('2019-04-01 00:00:00', freq='D'),
  'Shazam!'],
 

In [18]:
godzilla = build("Godzilla: King of the Monsters", '2019-05-31', '2020-05-31')
godzilla

2019-06-03 00:00:00
2019-06-24 00:00:00
2019-08-04 00:00:00
2019-08-15 00:00:00
2019-11-10 00:00:00
2019-11-22 00:00:00
2019-12-20 00:00:00
2020-02-24 00:00:00


[['40%',
  'Coming',
  '203',
  'Coming',
  Timestamp('2019-05-31 00:00:00', freq='D'),
  'Godzilla: King of the Monsters'],
 ['40%',
  'Coming',
  '217',
  'Coming',
  Timestamp('2019-06-01 00:00:00', freq='D'),
  'Godzilla: King of the Monsters'],
 ['39%',
  'Coming',
  '226',
  'Coming',
  Timestamp('2019-06-02 00:00:00', freq='D'),
  'Godzilla: King of the Monsters'],
 ['40%',
  'Coming',
  '254',
  'Coming',
  Timestamp('2019-06-04 00:00:00', freq='D'),
  'Godzilla: King of the Monsters'],
 ['39%',
  'Coming',
  '270',
  'Coming',
  Timestamp('2019-06-05 00:00:00', freq='D'),
  'Godzilla: King of the Monsters'],
 ['40%',
  '85%',
  '274',
  '15,079',
  Timestamp('2019-06-06 00:00:00', freq='D'),
  'Godzilla: King of the Monsters'],
 ['40%',
  '85%',
  '281',
  '15,749',
  Timestamp('2019-06-07 00:00:00', freq='D'),
  'Godzilla: King of the Monsters'],
 ['40%',
  '85%',
  '285',
  '16,990',
  Timestamp('2019-06-08 00:00:00', freq='D'),
  'Godzilla: King of the Monsters'],
 ['40%',


In [20]:
arr = [captain, joker, lion, aladdin, shazam, godzilla]
dfs = []

for movie in arr:
    dfs.append(pd.DataFrame(movie, columns=["critic", "audience", "criticNum", "audienceNum", "date", "film"]))

In [21]:
pd.concat(dfs).to_csv('all_films_wayback_2019_reviews.csv',index=False)